In [ ]:
%reload_ext autoreload
%autoreload 2


import sys, os

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np

from tqdm.notebook import tqdm

from xgboost import XGBRegressor, XGBClassifier
from econml.dml import NonParamDML, LinearDML
from econml.dr import LinearDRLearner, ForestDRLearner
from econml.metalearners import XLearner
from econml import metalearners

import sklearn
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR

from src.data.data_module import (
    generate_data_exp,
    _generate_covariates,
    _get_twins_data_covariates,
)
from src.data.utils import split_eval_cate

In [ ]:
def get_regressor(missing_value):
    return XGBRegressor(missing=missing_value, eval_metric="logloss")


def get_classifier(missing_value):
    return XGBClassifier(
        use_label_encoder=False, missing=missing_value, eval_metric="logloss"
    )


def get_imputer(missing_value):
    return IterativeImputer(
        max_iter=1500, tol=15e-4, random_state=None, missing_values=missing_value
    )
    # return SimpleImputer(missing_values=0, strategy='mean')


learners = {
    "T": lambda missing_value: metalearners.TLearner(
        models=get_regressor(missing_value)
    ),
    "X": lambda missing_value: XLearner(
        models=get_regressor(missing_value),
        propensity_model=get_classifier(missing_value),
        cate_models=get_regressor(missing_value),
    ),
    "DR": lambda missing_value: ForestDRLearner(
        model_propensity=get_classifier(missing_value),
        model_regression=get_regressor(missing_value),
    ),
}


def evaluate(ground_truth, estimate, W):
    PEHE = np.sqrt(((estimate - ground_truth) ** 2).mean())
    PEHE_0 = np.sqrt(((estimate[W == 0] - ground_truth[W == 0]) ** 2).mean())
    PEHE_1 = np.sqrt(((estimate[W == 1] - ground_truth[W == 1]) ** 2).mean())
    return PEHE, PEHE_0, PEHE_1

In [ ]:
# EXP. SETTINGS
runs = 100
sims = 10

data = "twins"

assert runs % sims == 0


train_size = 5000

d = 48  # twins
z_d_dim = 24
amount_of_missingness = 0.1
missing_value = -1

learner = "X"


# DEBUG SETTINGS
verbose = False

In [ ]:
ground_truth_cate = []
ground_truth_cate_0 = []
ground_truth_cate_1 = []


PEHE_impute_all = []
PEHE_impute_nothing = []
PEHE_impute_smartly = []
PEHE_impute_wrongly = []

effect_impute_all = []
effect_impute_nothing = []
effect_impute_smartly = []
effect_impute_wrongly = []

effect_impute_all_0 = []
effect_impute_nothing_0 = []
effect_impute_smartly_0 = []
effect_impute_wrongly_0 = []

effect_impute_all_1 = []
effect_impute_nothing_1 = []
effect_impute_smartly_1 = []
effect_impute_wrongly_1 = []
for _ in tqdm(range(sims)):
    X, X_, Y0, Y1, Y, CATE, W, Z_up, Z_down = generate_data_exp(
        train_size * 2,
        d,
        z_d_dim,
        amount_of_missingness,
        missing_value=missing_value,
        data=data,
    )

    assert 10 < train_size < len(X)

    for _ in tqdm(range(int(runs / sims)), leave=False):
        idxs = np.random.choice(range(len(X)), size=train_size, replace=False)
        include_idx = set(idxs)
        mask = np.array([(i in include_idx) for i in range(len(X))])

        X_train, Y_train, W_train, CATE_train = X_[mask], Y[mask], W[mask], CATE[mask]
        X_test, Y_test, W_test, CATE_test = X_[~mask], Y[~mask], W[~mask], CATE[~mask]

        ground_truth_cate.append(CATE_test)
        ground_truth_cate_0.append(CATE_test[W_test == 0])
        ground_truth_cate_1.append(CATE_test[W_test == 1])

        # IMPUTE ALL
        imputer = get_imputer(missing_value)
        imputer.fit(X_train)
        X_train_preprocessed = imputer.transform(X_train)
        X_test_preprocessed = imputer.transform(X_test)

        est_impute_all = learners[learner](missing_value)
        est_impute_all.fit(Y_train, W_train, X=X_train_preprocessed)

        # PEHE_impute_all.append(evaluate(CATE_test, te, W_test))
        effect_impute_all.append(est_impute_all.effect(X_test_preprocessed))
        effect_impute_all_0.append(
            est_impute_all.effect(X_test_preprocessed[W_test == 0])
        )
        effect_impute_all_1.append(
            est_impute_all.effect(X_test_preprocessed[W_test == 1])
        )

        if verbose:
            print("all", X_train.min())

        # IMPUTE NOTHING
        treatment_effects_impute_nothing = []
        X_train_preprocessed = X_train.copy()
        X_test_preprocessed = X_test.copy()

        est_impute_nothing = learners[learner](missing_value)
        est_impute_nothing.fit(Y_train, W_train, X=X_train_preprocessed)

        # PEHE_impute_nothing.append(evaluate(CATE_test, te, W_test))
        effect_impute_nothing.append(est_impute_nothing.effect(X_test_preprocessed))
        effect_impute_nothing_0.append(
            est_impute_nothing.effect(X_test_preprocessed[W_test == 0])
        )
        effect_impute_nothing_1.append(
            est_impute_nothing.effect(X_test_preprocessed[W_test == 1])
        )

        if verbose:
            print("nothing", X_train.min())

        # IMPUTE SMARTLY
        treatment_effects_impute_smartly = []
        imputer_smart = get_imputer(missing_value)
        imputer_smart.fit(X_train[:, z_d_dim:])

        X_train_preprocessed = X_train.copy()
        X_test_preprocessed = X_test.copy()

        X_train_preprocessed[:, z_d_dim:] = imputer_smart.transform(
            X_train[:, z_d_dim:]
        )
        X_test_preprocessed[:, z_d_dim:] = imputer_smart.transform(X_test[:, z_d_dim:])

        est_impute_smartly = learners[learner](missing_value)
        est_impute_smartly.fit(Y_train, W_train, X=X_train_preprocessed)

        # PEHE_impute_smartly.append(evaluate(CATE_test, te, W_test))
        effect_impute_smartly.append(est_impute_smartly.effect(X_test_preprocessed))
        effect_impute_smartly_0.append(
            est_impute_smartly.effect(X_test_preprocessed[W_test == 0])
        )
        effect_impute_smartly_1.append(
            est_impute_smartly.effect(X_test_preprocessed[W_test == 1])
        )

        if verbose:
            print("smart down", X_train[:, :z_d_dim].min())
            print("smart up", X_train[:, z_d_dim:].min())

        # IMPUTE WRONGLY
        treatment_effects_impute_wrongly = []
        imputer_wrongly = get_imputer(missing_value)
        imputer_wrongly.fit(X_train[:, :z_d_dim])

        X_train_preprocessed = X_train.copy()
        X_test_preprocessed = X_test.copy()

        X_train_preprocessed[:, :z_d_dim] = imputer_wrongly.transform(
            X_train[:, :z_d_dim]
        )
        X_test_preprocessed[:, :z_d_dim] = imputer_wrongly.transform(
            X_test[:, :z_d_dim]
        )

        est_impute_wrongly = learners[learner](missing_value)
        est_impute_wrongly.fit(Y_train, W_train, X=X_train_preprocessed)

        # PEHE_impute_wrongly.append(evaluate(CATE_test, te, W_test))
        effect_impute_wrongly.append(est_impute_wrongly.effect(X_test_preprocessed))
        effect_impute_wrongly_0.append(
            est_impute_wrongly.effect(X_test_preprocessed[W_test == 0])
        )
        effect_impute_wrongly_1.append(
            est_impute_wrongly.effect(X_test_preprocessed[W_test == 1])
        )

        if verbose:
            print("wrong down", X_train[:, :z_d_dim].min())
            print("wrong up", X_train[:, z_d_dim:].min())

In [ ]:
all_means, all_stds = split_eval_cate(effect_impute_all, ground_truth_cate, sims)
no_means, no_stds = split_eval_cate(effect_impute_nothing, ground_truth_cate, sims)
smart_means, smart_stds = split_eval_cate(
    effect_impute_smartly, ground_truth_cate, sims
)
wrong_means, wrong_stds = split_eval_cate(
    effect_impute_wrongly, ground_truth_cate, sims
)

all_means_0, all_stds_0 = split_eval_cate(
    effect_impute_all_0, ground_truth_cate_0, sims
)
no_means_0, no_stds_0 = split_eval_cate(
    effect_impute_nothing_0, ground_truth_cate_0, sims
)
smart_means_0, smart_stds_0 = split_eval_cate(
    effect_impute_smartly_0, ground_truth_cate_0, sims
)
wrong_means_0, wrong_stds_0 = split_eval_cate(
    effect_impute_wrongly_0, ground_truth_cate_0, sims
)

all_means_1, all_stds_1 = split_eval_cate(
    effect_impute_all_1, ground_truth_cate_1, sims
)
no_means_1, no_stds_1 = split_eval_cate(
    effect_impute_nothing_1, ground_truth_cate_1, sims
)
smart_means_1, smart_stds_1 = split_eval_cate(
    effect_impute_smartly_1, ground_truth_cate_1, sims
)
wrong_means_1, wrong_stds_1 = split_eval_cate(
    effect_impute_wrongly_1, ground_truth_cate_1, sims
)

const = np.append(all_means, [no_means, smart_means, wrong_means]).max()
const_0 = np.append(all_means_0, [no_means_0, smart_means_0, wrong_means_0]).max()
const_1 = np.append(all_means_1, [no_means_1, smart_means_1, wrong_means_1]).max()

print("# SETUP")
print(f"# DATA = {data}")
print(f"# learner = {learner}")
print(f"# amount_of_missingness = {amount_of_missingness}")
print(f"# z_d_dim = {z_d_dim}")
print(f"# train size = {train_size}")
print(f"# amount of runs = {runs}")

print(f"#   ALL IMPUTATION  :\t{np.mean(all_means)/const}\t{np.mean(all_stds)/const}")
print(f"#   NO IMPUTATION   :\t{np.mean(no_means)/const}\t{np.mean(no_stds)/const}")
print(
    f"#   SMART IMPUTATION:\t{np.mean(smart_means/const)}\t{np.mean(smart_stds)/const}"
)
print(
    f"#   WRONG IMPUTATION:\t{np.mean(wrong_means/const)}\t{np.mean(wrong_stds)/const}"
)
print(f"#   ---------------- PEHE_0")
print(
    f"#   ALL IMPUTATION  :\t{np.mean(all_means_0)/const_0}\t{np.mean(all_stds_0)/const_0}"
)
print(
    f"#   NO IMPUTATION   :\t{np.mean(no_means_0)/const_0}\t{np.mean(no_stds_0)/const_0}"
)
print(
    f"#   SMART IMPUTATION:\t{np.mean(smart_means_0)/const_0}\t{np.mean(smart_stds_0)/const_0}"
)
print(
    f"#   WRONG IMPUTATION:\t{np.mean(wrong_means_0)/const_0}\t{np.mean(wrong_stds_0)/const_0}"
)
print(f"#   ---------------- PEHE_1")
print(
    f"#   ALL IMPUTATION  :\t{np.mean(all_means_1)/const_1}\t{np.mean(all_stds_1)/const_1}"
)
print(
    f"#   NO IMPUTATION   :\t{np.mean(no_means_1)/const_1}\t{np.mean(no_stds_1)/const_1}"
)
print(
    f"#   SMART IMPUTATION:\t{np.mean(smart_means_1)/const_1}\t{np.mean(smart_stds_1)/const_1}"
)
print(
    f"#   WRONG IMPUTATION:\t{np.mean(wrong_means_1)/const_1}\t{np.mean(wrong_stds_1)/const_1}"
)

In [ ]:
# SETUP
# DATA = twins
# learner = T
# amount_of_missingness = 0.1
# z_d_dim = 24
# train size = 5000
# amount of runs = 100
#   ALL IMPUTATION  :	0.7572275405984663	0.10137960558508072
#   NO IMPUTATION   :	0.7393209967553888	0.295306071041042
#   SMART IMPUTATION:	0.7289547289934614	0.30209381978963085
#   WRONG IMPUTATION:	0.7591579245817164	0.0955984508262786
#   ---------------- PEHE_0
#   ALL IMPUTATION  :	0.9590341663355869	0.06496131114974854
#   NO IMPUTATION   :	0.7451106135761542	0.07975262667674156
#   SMART IMPUTATION:	0.7407708927821102	0.0768721375689943
#   WRONG IMPUTATION:	0.9570022721829745	0.06626924375357977
#   ---------------- PEHE_1
#   ALL IMPUTATION  :	0.7152607889541894	0.1098621950516329
#   NO IMPUTATION   :	0.7188177477139469	0.31928301804139647
#   SMART IMPUTATION:	0.7086193946113668	0.32730764420330927
#   WRONG IMPUTATION:	0.7175722471726768	0.10473280009343697


# SETUP
# DATA = twins
# learner = DR
# amount_of_missingness = 0.1
# z_d_dim = 24
# train size = 5000
# amount of runs = 100
#   ALL IMPUTATION  :	0.06992683728470135	0.12223030320868446
#   NO IMPUTATION   :	0.0015434309157597002	0.003057462629854516
#   SMART IMPUTATION:	0.001306512505831117	0.01568340567519754
#   WRONG IMPUTATION:	0.1441742963798253	0.3643103653106824
#   ---------------- PEHE_0
#   ALL IMPUTATION  :	0.07927710237670467	0.11648451860355706
#   NO IMPUTATION   :	0.0026353635492602787	0.004915892409350565
#   SMART IMPUTATION:	0.002893320158420537	0.02810305270088261
#   WRONG IMPUTATION:	0.15222514046801144	0.3761636634413026
#   ---------------- PEHE_1
#   ALL IMPUTATION  :	0.06953947911448746	0.12268835905442628
#   NO IMPUTATION   :	0.0015005056999507465	0.0029856093072289295
#   SMART IMPUTATION:	0.000083647382212736	0.0151963668453146
#   WRONG IMPUTATION:	0.14383829666214232	0.36385829039729634

# SETUP
# DATA = twins
# learner = X
# amount_of_missingness = 0.1
# z_d_dim = 24
# train size = 5000
# amount of runs = 100
#   ALL IMPUTATION  :	0.8971831019382526	0.06530186137937415
#   NO IMPUTATION   :	0.9191404516804176	0.11422708700128295
#   SMART IMPUTATION:	0.8868597024682495	0.11941180975316847
#   WRONG IMPUTATION:	0.9067510558077473	0.06297707246029788
#   ---------------- PEHE_0
#   ALL IMPUTATION  :	0.9191401999862722	0.07143898537490642
#   NO IMPUTATION   :	0.7557023057654503	0.06678072605997161
#   SMART IMPUTATION:	0.7406099093284292	0.0643811955837617
#   WRONG IMPUTATION:	0.9139090855281191	0.06852342110040915
#   ---------------- PEHE_1
#   ALL IMPUTATION  :	0.8803944617584534	0.06673896528555705
#   NO IMPUTATION   :	0.9090887301221944	0.12087188543240905
#   SMART IMPUTATION:	0.8700209267752238	0.12665969456825846
#   WRONG IMPUTATION:	0.8810791883919776	0.0646153150283087